<a href="https://colab.research.google.com/github/talitacardenas/The_Bridge_School_DataScience_PT/blob/main/soluci%C3%B3n_Pra1_PostgreSQL_Python_Talita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%shell
import psycopg2
psycopg2.__version__

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'2.7.6.1 (dt dec pq3 ext lo64)'

In [2]:
# Connect to an existing database HEROKU _ vuestro nombre
conn = psycopg2.connect(
    host="ec2-54-216-185-51.eu-west-1.compute.amazonaws.com",
    database="d1ifct7i405p1j",
    user="crilkwrqvnmtlq",
    password="c11b80adb4793d006c2bd31fc39c4ef546a26481c8d179f3a183fcc9b49d6ce3",
    port=5432)

In [3]:
query = """
SELECT * 
FROM web_events 
LIMIT 15;
"""

In [4]:
import pandas as pd

In [5]:
pd.read_sql(con=conn,sql=query)

,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct
3,4,1001,2016-01-02 00:55:03,direct
4,5,1001,2016-02-01 19:02:33,direct
5,6,1001,2016-03-02 15:15:22,direct
6,7,1001,2016-04-01 10:58:55,direct
7,8,1001,2016-05-01 15:26:44,direct
8,9,1001,2016-05-31 20:53:47,direct
9,10,1001,2016-06-30 12:09:45,direct


***
1. ¿Tenemos actores en la tabla de actores que comparten el nombre completo y, en caso afirmativo, muestran esos nombres compartidos?

In [6]:
query = '''
SELECT COUNT(DISTINCT first_name || last_name)
FROM actor
'''

In [7]:
pd.read_sql(con=conn, sql=query)

,count
0,199


In [8]:
query_alt1 = '''
SELECT CONCAT(first_name,' ', last_name) Nombre_completo, COUNT('Nombre_completo') Counter
FROM actor
GROUP BY 1
HAVING COUNT('Nombre_completo') >1;
'''

In [ ]:
pd.read_sql(con=conn, sql=query)

,count
0,199


***
2. Muestra los nombres de los clientes que comparten la misma dirección (por ejemplo, esposo y esposa).

In [ ]:
query = '''
SELECT c1.first_name, c1.last_name, c2.first_name, c2.last_name
FROM customer c1 JOIN customer c2
ON c1.customer_id <> c2.customer_id AND c1.address_id = c2.address_id
'''

In [ ]:
pd.read_sql(con=conn, sql=query)

,first_name,last_name,first_name,last_name


***
3. Muestra el monto total pagado por todos los clientes en la tabla de pagos.

In [9]:
query = '''
SELECT SUM(amount)
from payment
'''

In [10]:
pd.read_sql(con=conn, sql=query)

,sum
0,61312.04


***
4. Muestre el monto total pagado por cada cliente en la tabla de pagos

In [11]:
query = '''
SELECT customer_id, SUM(amount)
from payment
group by customer_id
order by customer_id
'''

In [12]:
pd.read_sql(con=conn, sql=query)

,customer_id,sum
0,1,114.70
1,2,123.74
2,3,130.76
3,4,81.78
4,5,134.65
...,...,...
594,595,110.71
595,596,73.78
596,597,87.77
597,598,83.78


***
5. ¿Cuál es el pago total más alto realizado?


In [13]:
query = '''
SELECT SUM(amount) AS tot_payments
FROM payment
GROUP BY customer_id
ORDER BY tot_payments DESC
LIMIT 1

'''

In [14]:
pd.read_sql(con=conn, sql=query)

,tot_payments
0,211.55


In [ ]:
query = '''
SELECT
 customer_id, amount 
FROM
  payment
WHERE
  amount = 
  (SELECT
    MAX(amount)
    FROM
    payment
  )
ORDER BY 2 DESC
LIMIT 10
'''

In [ ]:
pd.read_sql(con=conn, sql=query)

,customer_id,amount
0,362,11.99
1,204,11.99
2,116,11.99
3,195,11.99
4,237,11.99
5,591,11.99
6,592,11.99
7,13,11.99


***
6. 6. ¿Cuál es el nombre del cliente que realizó los pagos totales más altos?

In [ ]:
query = '''
SELECT first_name, last_name
FROM customer
WHERE customer_id IN
  (SELECT customer_id
  FROM payment
  GROUP BY customer_id
  HAVING SUM(amount) =
    (SELECT SUM(amount)
    FROM payment
    GROUP BY customer_id
    ORDER BY SUM(amount) DESC LIMIT 1
    )
  )
'''

In [ ]:
pd.read_sql(con=conn, sql=query)

,first_name,last_name
0,Eleanor,Hunt


***
7. 7. ¿Cuáles son las películas que más se alquilaron?

In [ ]:
query_1 = '''
SELECT
 film.title, COUNT(rental.rental_id )
FROM
  rental
JOIN
  inventory
ON
  rental.inventory_id = inventory.inventory_id
JOIN
  film
ON
  film.film_id = inventory.film_id
GROUP BY
  1
ORDER BY 
  2 DESC
LIMIT (11); /* ¿11? */
'''

In [ ]:
pd.read_sql(con=conn, sql=query_1)

,title,count
0,Bucket Brotherhood,34
1,Rocketeer Mother,33
2,Forward Temple,32
3,Ridgemont Submarine,32
4,Grit Clockwork,32
5,Juggler Hardly,32
6,Scalawag Duck,32
7,Apache Divine,31
8,Robbers Joon,31
9,Wife Turn,31


In [ ]:
query = '''
SELECT film_id, count(film_id)
FROM rental r
JOIN inventory i
ON r.inventory_id = i.inventory_id
GROUP BY film_id
ORDER BY count(film_id) DESC
'''

In [ ]:
pd.read_sql(con=conn, sql=query)

,film_id,count
0,103,34
1,738,33
2,730,32
3,382,32
4,767,32
...,...,...
953,459,5
954,107,5
955,400,4
956,904,4


***
8. 8. Qué películas se han alquilado hasta ahora.

In [ ]:
query = '''
SELECT count(title)
FROM film
WHERE film_id IN
  (SELECT distinct film_id
  FROM rental JOIN inventory
  ON rental.inventory_id = inventory.inventory_id)
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,count
0,958


***
9. 9. Qué películas no se han alquilado hasta ahora.

In [ ]:
query = '''
SELECT title
FROM film
WHERE film_id NOT IN
  (SELECT DISTINCT film_id
  FROM rental JOIN inventory
  ON rental.inventory_id = inventory.inventory_id)
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,title
0,Alice Fantasia
1,Apollo Teen
2,Argonauts Town
3,Ark Ridgemont
4,Arsenic Independence
5,Boondock Ballroom
6,Butch Panther
7,Catch Amistad
8,Chinatown Gladiator
9,Chocolate Duck


***
10. Qué clientes no han alquilado ninguna película hasta ahora.

In [ ]:
query = '''
SELECT first_name, last_name
FROM customer c
WHERE NOT EXISTS
  (SELECT DISTINCT customer_id
  FROM rental r
  WHERE c.customer_id = r.customer_id)

'''

In [15]:
pd.read_sql(con=conn,sql=query)

,tot_payments
0,211.55


11.Muestra cada película y la cantidad de veces que se alquila.

In [16]:
query = '''
select film.title, COUNT(rental.rental_id )
from rental
join inventory
on rental.inventory_id = inventory.inventory_id
join film
on film.film_id = inventory.film_id
group by 1 
order by 1 asc 
'''

In [18]:
pd.read_sql(con=conn,sql=query)

,title,count
0,Academy Dinosaur,23
1,Ace Goldfinger,7
2,Adaptation Holes,12
3,Affair Prejudice,23
4,African Egg,12
...,...,...
953,Young Language,7
954,Youth Kick,6
955,Zhivago Core,9
956,Zoolander Fiction,17


12.  Muestra la cantidad de películas en las que actuó cada actor.

In [20]:
query = '''
select concat(actor.first_name,' ',actor.last_name),
count(film.film_id)
from film
join film_actor 
on film_actor.film_id = film.film_id 
join actor
on actor.actor_id = film_actor.actor_id 
group by 1
order by 2 desc
'''

In [21]:
pd.read_sql(con=conn,sql=query)

,concat,count
0,Susan Davis,54
1,Gina Degeneres,42
2,Walter Torn,41
3,Mary Keitel,40
4,Matthew Carrey,39
...,...,...
194,Sissy Sobieski,18
195,Julia Zellweger,16
196,Julia Fawcett,15
197,Judy Dean,15


***
13. Muestre los nombres de los actores que actuaron en más de 20 películas.

In [ ]:
query = '''
SELECT first_name, last_name, COUNT(film_actor.film_id)
FROM film_actor JOIN actor
ON film_actor.actor_id = actor.actor_id
GROUP BY (first_name, last_name)
HAVING COUNT(film_actor.film_id) > 20
ORDER BY 3 DESC
'''

In [22]:
pd.read_sql(con=conn,sql=query)

,concat,count
0,Susan Davis,54
1,Gina Degeneres,42
2,Walter Torn,41
3,Mary Keitel,40
4,Matthew Carrey,39
...,...,...
194,Sissy Sobieski,18
195,Julia Zellweger,16
196,Julia Fawcett,15
197,Judy Dean,15


14. ¿Cuántos actores tienen 8 letras solo en sus nombres?

In [24]:
query = '''
select count(first_name)
from actor a 
where length(first_name) > 8
'''

In [25]:
pd.read_sql(con=conn,sql=query)

,count
0,6


15. Para todas las películas clasificadas como "PG", muéstrame la película y la cantidad de veces que se alquiló.
*Escribir una subconsulta para seleccionar todos los film_ids que se han alquilado hasta ahora. Para eso, se necesita unir tablas de alquiler e inventario en la columna común Inventory_id.*


16. Muestra las películas que se ofrecen para alquilar en store_id 1 y no se ofrecen en store_id 2.

17. Muestre las películas que se ofrecen para alquilar en cualquiera de las dos tiendas 1 y 2.

18. Muestra los títulos de las películas que se ofrecen en ambas tiendas al mismo tiempo

19. Para cada tienda, muestre el número de clientes que son miembros de esa tienda

20. Muestra el título de la película más alquilada en la tienda con store_id 1



```
# Tiene formato de código
```

***
22. Muestre los customer_id para aquellos clientes que alquilaron un DVD de películas más de una vez.

In [ ]:
query = '''
SELECT
 rental.customer_id, inventory.film_id, COUNT(rental.rental_id)
FROM
  rental
JOIN
  inventory
ON
  inventory.inventory_id = rental.inventory_id
WHERE rental.customer_id = 314
GROUP BY
  1,2
HAVING COUNT(rental.rental_id) > 1
ORDER BY 3 DESC;
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,customer_id,film_id,count
0,314,111,3
1,314,973,2


In [ ]:
query = '''
WITH temp as
(
  SELECT rental_id, rental_date, customer_id, film_id
  FROM rental JOIN inventory
  ON rental.inventory_id = inventory.inventory_id
  )
SELECT t1.customer_id, count(t1.film_id)
FROM temp t1
JOIN temp t2 
ON t1.customer_id = t2.customer_id AND t1.film_id = t2.film_id AND t1.rental_date <> t2.rental_date
GROUP BY t1.customer_id
HAVING COUNT(t1.film_id) > 1
ORDER BY 2 DESC;
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,customer_id,count
0,122,12
1,314,8
2,376,6
3,30,6
4,482,6
...,...,...
175,90,2
176,99,2
177,108,2
178,111,2


***
27. 27. Busque el nombre del actor que comience con "P" seguido de (una e o una a) y luego cualquier otra letra

In [ ]:
query = '''
SELECT DISTINCT
  first_name
FROM
  actor
WHERE
  first_name LIKE 'Pe%' OR first_name LIKE 'Pa%';
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,first_name
0,Parker
1,Penelope


In [ ]:
query = '''
SELECT first_name, last_name
FROM actor
WHERE first_name SIMILAR TO 'P(a|e)%';
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,first_name,last_name
0,Penelope,Guiness
1,Parker,Goldberg
2,Penelope,Pinkett
3,Penelope,Cronyn
4,Penelope,Monroe


In [ ]:
query = '''
SELECT first_name, last_name
FROM actor
WHERE first_name SIMILAR TO 'P[a-zA-Z]{5,}';
'''

In [ ]:
pd.read_sql(con=conn,sql=query)

,first_name,last_name
0,Penelope,Guiness
1,Parker,Goldberg
2,Penelope,Pinkett
3,Penelope,Cronyn
4,Penelope,Monroe


***